In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pywt

# Frame Extraction

In [3]:
import cv2
import os

# Function to extract frames from video
def extract_frames(video_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    video_capture = cv2.VideoCapture(video_path)
    if not video_capture.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    frame_count = 0
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.png")
        cv2.imwrite(frame_filename, frame)
        print(f"Extracted frame {frame_count}")
        frame_count += 1
    video_capture.release()
    print("Frame extraction completed.")

extract_frames('testVid.mp4', 'extracted_frames')


Extracted frame 0
Extracted frame 1
Extracted frame 2
Extracted frame 3
Extracted frame 4
Extracted frame 5
Extracted frame 6
Extracted frame 7
Extracted frame 8
Extracted frame 9
Extracted frame 10
Extracted frame 11
Extracted frame 12
Extracted frame 13
Extracted frame 14
Extracted frame 15
Extracted frame 16
Extracted frame 17
Extracted frame 18
Extracted frame 19
Extracted frame 20
Extracted frame 21
Extracted frame 22
Extracted frame 23
Extracted frame 24
Extracted frame 25
Extracted frame 26
Extracted frame 27
Extracted frame 28
Extracted frame 29
Extracted frame 30
Extracted frame 31
Extracted frame 32
Extracted frame 33
Extracted frame 34
Extracted frame 35
Extracted frame 36
Extracted frame 37
Extracted frame 38
Extracted frame 39
Extracted frame 40
Extracted frame 41
Extracted frame 42
Extracted frame 43
Extracted frame 44
Extracted frame 45
Extracted frame 46
Extracted frame 47
Extracted frame 48
Extracted frame 49
Extracted frame 50
Extracted frame 51
Extracted frame 52
Ext

# Creating a Watermark

In [ ]:
def create_watermark(shape):
    return np.random.randint(0, 256, shape, dtype=np.uint8)

rw, cw = (10, 10)  
W = create_watermark((rw, cw))
plt.imshow(W, cmap=plt.cm.gray)

# Initial Parameters

In [10]:
a, b = 5, 5  
c1, c2 = 0.5, 0.5 
Z1, Z2 = 0.5 + 0.5j, 0.5 + 0.5j
T = 1

# Helper Functions


In [23]:
def dwt_transform(image, wavelet = 'haar'):
    
    b, g, r = cv2.split(image)

    coeffsb = pywt.dwt2(b, wavelet)
    coeffsg = pywt.dwt2(g, wavelet)
    coeffsr = pywt.dwt2(r, wavelet)
    
    # only returning LL band with 3 channels
    return cv2.merge((coeffsb[0], coeffsg[0], coeffsr[0]))

In [31]:
def CFold_1(Z):
        return Z.real % 1 + (Z.imag % 1) * 1j

def compute_Z1(a, c1, Z1, Z2):
    return CFold_1(a * ((Z1 / Z2)**2)) + c1 

def compute_Z2(b, c2, Z1, Z2):    
    return CFold_1(b * ((Z2 / Z1)**2)) + c2 

In [32]:
def svd(matrix, inverse = False):
    if inverse:
        pass
    return np.linalg.svd(matrix)

In [33]:
def condition_for_zero(Wij, U, T):
    return (Wij == 0  and 
            np.abs(U[0, 0]) > np.abs(U[1, 0]) and 
            np.abs(U[1, 0]) - np.abs(U[0, 0]) < T)

In [34]:
def condition_for_one(Wij, U, T):
    return (Wij == 1 and  
            np.abs(U[0, 0]) < np.abs(U[1, 0]) and 
            np.abs(U[1, 0]) - np.abs(U[0, 0]) > T)

# Function

In [ ]:
def embedding_extraction_function(I, W, a, b, c1, c2, Z1, Z2,  extract = False):

    LLband = dwt_transform(I)    
    rc, cc, channels = LLband.shape

    rcB4, ccB4 = rc / 4, cc / 4
    mask = np.zeros((rcB4, ccB4, channels)) 
    
    for i in range(1,rw):
        
        for j in range(1,cw):
            
            while True:
                
                Z1 = compute_Z1(a, c1, Z1, Z2)
                Z2 = compute_Z2(b, c2, Z1, Z2)
    
                x = np.floor(Z1.real * (10**14)) % rcB4
                y = np.floor(Z1.imag * (10**14)) % ccB4
    
                channel = np.floor(Z2.real * (10**14)) % channels
    
                B = np.floor(Z2.imag * (10**14)) % 2
                if mask[x+1 , y+1, channel] == 0:
                    break

        block = LLband[x * 4 + 1 : x * 4 + 4, 
                       y * 4 + 1 : y * 4 + 4,
                       channel]
        
        U, S, V = svd(block)

        if extract == False:
            
            # mean of first two elements of first vector in U
            meanV = (np.abs(U[0, 0]) + np.abs(U[1, 0])) / 2

            #WaterMarking
            if condition_for_zero(W[i, j], U, T):
                    U[0, 0] = np.sign(U[0, 0]) * (meanV + T / 2)
                    U[1, 0] = np.sign(U[1, 0]) * (meanV - T / 2)

            # XOR
            # if Wij != B then Wij = 1 and block below executes
            W[i, j] = W[i,j] ^ B 
            
            if : # defin T later
                    U[1,1] = np.sign(U[1,1]) * (meanV - T/2)
                    U[2,1] = np.sign(U[2,1]) * (meanV + T/2)

            #Reconstruction
            rank = S.size
            recon = U[:,:rank]@np.diag(S)@V[:rank,:]
            if channel == 0:
                rLL[x*4+1: x*4+4, y*4+1: y*4+4] = recon
            elif channel == 1:
                gLL[x*4+1: x*4+4, y*4+1: y*4+4] = recon
            else:
                bLL[x*4+1: x*4+4, y*4+1: y*4+4] = recon

        #EXTRACTION
        if type == 1:
            if np.abs(U[1,1]) > np.abs(U[2,1]):
                EW[i,j] = 0
            else:
                EW[i,j] = 1
            EW[i,j] = EW[i,j] ^ bita #DECRYPTION PROCESS
        mask[x+1,y+1,channel] = 1
    
    #Save final Z1, Z2 for initial conditions of next frame
    if type == 0:
        WIr = pywt.idwt2(recon, rLH,rHL,rHH)
        WIg = pywt.idwt2(recon, gLH,gHL,gHH)
        WIb = pywt.idwt2(recon, bLH,bHL,bHH)
        WI = np.array([WIr,WIg,WIb])
        return WI
    else:
        return EW

        